# Tutorial 08 - OEM Files and Covariance

In version 0.9.0, ANISE introduces the ability to read CCSDS OEM (Orbit Ephemeris Message) files and interpolate them, including covariance data. This tutorial demonstrates:

1.  **Reading an OEM File:** Loading an OEM file into an `Ephemeris` object.
2.  **Interpolation:** Querying state and covariance at arbitrary times.
3.  **Covariance Frames:** Accessing covariance in different frames (RIC, Inertial, etc.).
4.  **Event Integration:** Combining OEM data with event finding to query covariance at specific event times.

In [1]:
from pathlib import Path
from anise import Almanac
from anise.astro import Ephemeris, LocalFrame, DataType, Orbit, Frame
from anise.constants import Frames
from anise.time import Epoch, Unit
from anise.analysis import Event, StateSpec, FrameSpec

# Load a base Almanac (for frame definitions)
almanac = Almanac("../../data/pck08.pca")
almanac.describe()

=== PLANETARY DATA #0: `../../data/pck08.pca` ===
┌───────┬─────────┬──────────────────────────┬─────────────────┬─────────────────┬─────────────────┬───────────────────────┬─────────────────────┬────────────────────────────────────────────────────────┐
│ Name  │ ID      │ Gravity param (km^3/s^2) │ Major axis (km) │ Minor axis (km) │ Polar axis (km) │ Pole right asc.       │ Pole declination    │ Prime meridian                                         │
├───────┼─────────┼───────────────────��──────┼─────────────────┼─────────────────┼─────────────────┼───────────────────────┼─────────────────────┼────────────────────────────────────────────────────────┤
│ Unset │ 1       │ 22031.78000000002        │ 2439.7          │ 2439.7          │ 2439.7          │ 281.01 + -0.033 t     │ 61.45 + -0.005 t    │ 329.548 + 6.1385025 t                                  │
├───────┼─────────┼──────────────────────────┼─────────────────┼─────────────────┼─────────────────┼───────────────────────┼─────────

## 1. Reading an OEM File

We can load an OEM file directly into an `Ephemeris` object. This object holds the interpolated data.

In [2]:
oem_path = "../../data/tests/ccsds/oem/LRO_Nyx.oem"
print(f"Loading OEM from {oem_path}")

ephem = Ephemeris.from_ccsds_oem_file(oem_path)
print("Ephemeris loaded:")
print(ephem)

(start, end) = ephem.domain()
print(f"Domain: {start} to {end}")

Loading OEM from ../../data/tests/ccsds/oem/LRO_Nyx.oem
Ephemeris loaded:
2010-LRO ephem from 2024-01-01T00:00:00 UTC to 2024-01-01T00:03:00 UTC (4 states, spans 3 min)
Domain: 2024-01-01T00:00:00 UTC to 2024-01-01T00:03:00 UTC


## 2. Interpolation and Covariance

We can query the covariance at any time within the domain. ANISE supports transforming the covariance into various local frames:
-   **RIC (Radial, In-track, Cross-track):** Also known as RTN.
-   **Inertial:** The frame in which the data is defined (usually J2000).
-   **VNC (Velocity, Normal, Co-normal):** Useful for maneuver planning.
-   **RCN:** Radial, Cross-track, Normal.

In [3]:
query_time = start + (end - start) * 0.5
print(f"Querying covariance at {query_time}")

# Covariance in RIC frame
covar_ric = ephem.covar_at(query_time, LocalFrame.RIC, almanac)
print("\nCovariance (RIC):")
print(covar_ric)

# Covariance in Inertial frame
covar_inertial = ephem.covar_at(query_time, LocalFrame.Inertial, almanac)
print("\nCovariance (Inertial):")
print(covar_inertial)

Querying covariance at 2024-01-01T00:01:30 UTC

Covariance (RIC):
Covariance in RIC
  ┌                                                             ┐
  │  0.962697  0.417577 -0.432062  0.004468  0.002043 -0.001881 │
  │  0.417577  0.672621  0.372849  0.002017  0.003435  0.001994 │
  │ -0.432062  0.372849  0.832859 -0.001914  0.001990  0.004048 │
  │  0.004468  0.002017 -0.001914  0.000059  0.000024 -0.000022 │
  │  0.002043  0.003435  0.001990  0.000024  0.000036  0.000020 │
  │ -0.001881  0.001994  0.004048 -0.000022  0.000020  0.000044 │
  └                                                             ┘



Covariance (Inertial):
Covariance in Inertial
  ┌                                                             ┐
  │  0.221927  0.425854  0.264891  0.001022  0.001972  0.001206 │
  │  0.425854  1.115708  0.013385  0.001966  0.005345 -0.000217 │
  │  0.264891  0.013385  1.138789  0.001213 -0.000234  0.005647 │
  │  0.001022  0.001966  0.001213  0.000011  0.000022  0.000013 │
  │  0.00

## 3. Combining with Event Finding

A powerful workflow is to find events using the `Almanac` and then inspect the covariance at those specific times.

First, we load the OEM into an `Almanac`. We must provide a NAIF ID for the object in the OEM file (e.g., LRO is -85).

In [6]:
lro_id = -85
# Load OEM into a new Almanac, assigning it the LRO ID
almanac_oem = Almanac.from_ccsds_oem_file(oem_path, lro_id)
# Load PCK for frame definitions (needed for event finding involving other bodies or frames)
almanac_oem = almanac_oem.load("../../data/pck08.pca")

print("Almanac created from OEM:")
almanac_oem.describe(spk=True)

ephemeris contains covariance, which is NOT copied to the SPICE BSP file


Almanac created from OEM:
=== SPK #0: `2025-12-29T21:07:33.292829705 UTC` ===
┌─────────────────────────────────────────┬────────────────┬────────────┬───────────────────────────────────┬───────────────────────────────────┬──────────┬──────────────────────┐
│ Name                                    │ Target         │ Center     │ Start epoch                       │ End epoch                         │ Duration │ Interpolation kind   │
├─────────────────────────────────────────┼────────────────┼───────────���┼───────────────────────────────────┼───────────────────────────────────┼──────────┼──────────────────────┤
│ 2010-LRO (converted by Nyx Space ANISE) │ body -85 J2000 │ Moon J2000 │ 2024-01-01T00:01:09.183898601 TDB │ 2024-01-01T00:04:09.183898727 TDB │ 3 min    │ Hermite Unequal Step │
└─────────────────────────────────────────┴────────────────┴────────────┴───────────────────────────────────┴───────────────────────────────────┴──────────┴──────────────────────┘


Now we define an event, for example, finding Apoapsis.

In [7]:
lro_frame = Frame(lro_id, 1) # Assuming J2000 orientation
moon_frame = Frames.MOON_J2000

lro_state_spec = StateSpec(
    target_frame=FrameSpec.Loaded(lro_frame),
    observer_frame=FrameSpec.Loaded(moon_frame),
    ab_corr=None,
)

apolune = Event.apoapsis()

print(f"Searching for apoapsis events...")
apo_events = almanac_oem.report_events(lro_state_spec, apolune, start, end)
print(f"Found {len(apo_events)} apoapsis events.")

Searching for apoapsis events...
Found 0 apoapsis events.


Finally, we iterate through the found events and query the covariance from the original `Ephemeris` object at each event time.

In [8]:
print("Querying covariance at event times:")
for i, event in enumerate(apo_events):
    event_time = event.orbit.epoch
    
    # Query covariance in RIC frame
    cov = ephem.covar_at(event_time, LocalFrame.RIC, almanac_oem)
    
    print(f"\nEvent {i+1} at {event_time}:")
    print(cov)

Querying covariance at event times:
